# puztool.logic_grid: Grid Logic helpers

In [1]:
from puztool import *
from puztool.logic_grid import *

These are tools for working with the sort of logic puzzles where you have a bunch of categories, a bunch of unique labels in each category, and a bunch of declarations about various subsets of these. For example:

-----

Five people (Brita, Galal, Sam, Violet, and Zork) all have different color hair (Blue, Green, Red, Taupe, and Violet) and were born under different astrological signs (Aries, Scorpio, Virgo, Crabbus, or Gahoolie, The Vase of Tulips). Each has a hairspeed of between one and four follicles/second. Additionally, the following are all true:

1. Brita is not an Aries
2. Sam, the Green haired person, and the Virgo are three different people.
3. Violet does not have Green hair.
3. The person with Blue hair is a Scorpio.
4. Brita has Red or Blue hair.
5. Neither Zork nor the Crabbus has Green hair.
6. The Violet-haired person has more letters in their name than the person born under Gahoolie, The Vase of Tulips.
6. The Red-haired person has twice the hairspeed of the Crabbus.
7. Violet has Taupe hair or is an Aries, but not both.
8. The total combined hairspeed of all persons is more than 6, but less than 10.

For each person, what color and speed of hair do they have, and what sign were they born under?

-----

We represent a simple grid of values with the `Grid` type, which takes as input a data frame containing all possible values for each category, in no particular order:

In [2]:
categories = dict(
    name = ['Brita', 'Galal', 'Sam', 'Violet', 'Zork'],
    color = ['Blue', 'Green', 'Red', 'Taupe', 'Violet'],
    sign = ['Aries', 'Scorpio', 'Virgo', 'Crabbus', 'Gahoolie']
)
g = Grid(categories)

This `Grid` object has internal grids of booleans indicating which pairs of values are known to be true or false; initially it's all `None`.

In [3]:
g.grids['name']['color']

array([[None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None]], dtype=object)

To set these values, we can use the helper methods `exclude`, `include`, and `requireOne`.

`exclude(*values)` takes any number of values and encodes that they are all mutually exclusive.
`include(*values)` indicates that the specified values all correspond to each other.
`requireOne(value, [options])` indicates that the first value corresponds to exactly one of the chosen options.

In all cases, you don't need to indicate which category your values are in - `Grid` will infer this from the value itself. But you can specify the category explicitly by `category:value`; this is necessary if the value is ambiguous (for example, `name:Violet` refers to the person named Violet, while `color:Violet` refers to the person with violet hair). Thus, we can encode many of the rules listed above with:

In [4]:
# Brita is not an Aries
g.exclude("Brita", "Aries") # Brita and Aries cannot be in the same row
# Sam, Green hair, and the Virgo are different people
g.exclude("Sam", "Green", "Virgo") # No two of these three can be in the same row
# Violet does not have green hair
g.exclude("name:Violet", "Green") # Violet (the name) cannot be in the same row as Green
# The blue-haired person is a scorpio
g.require("Scorpio", "Blue") # Scorpio and Blue must be in the same row
# Brita's hair is red or blue
g.requireOne("Brita", ["Red", "Blue"]) # The row with Brita must contain Red or Blue
# Neither Zork nor Crabbus has Green hair
g.exclude("Zork", "Crabbus", "Green") # None of these are in the same row

In an ideal world I'd build a tool for viewing this right here in the notebook, but this isn't that world so instead you can use `g.html_link()` to get a link to the grid-solving tool at http://www.jsingler.de/apps/logikloeser/, prefilled with your grid (use `g.get_link()` if you want plain text instead of a jupyter `HTML` object):

In [5]:
g.html_link()

Following the above link, you can see that the tool there has inferred a bunch of values for you, and you can probably solve the rest yourself using the last few rules. However, the grid isn't sophisticated enough to express constraints that aren't simple combinations of binaries, such as "The Violet-haired person has more letters in their name than the person born under Gahoolie, The Vase of Tulips.", and the grid can't even express `hairspeed`, so if you're just using `Grid` you'll have to solve that part yourself.

Alternatively, if you have [Numberjack](https://github.com/eomahony/Numberjack) installed, you can use the full solver.

## Solver

For more complicated setups, you'll want to use the `Solver` class, which is a subclass of `Grid` that exposes [Numberjack](https://github.com/eomahony/Numberjack) variables for each cell of the logic grids, and also supports additional non-exclusive categories and constraints.

The input to `Solver` is the same as to `Grid`, except it also takes a dict of `extra_categories`, which are categories that don't fit the grid pattern - i.e., they're not a set of mutually exclusive values, but rather are one value per row chosen from some domain. In this case, `hairspeed` can be 1, 2, 3, or 4:

In [6]:
s = Solver(categories, extra_categories=dict(hairspeed=[1,2,3,4]))

The solver maintains a grid of numberjack variables for all the intersections between different values, as well as a convenient mapping from extra categories to all associated variables:

In [7]:
# binary variable for "does violet have green hair"
print(s.vars['name:Violet', 'color:Green'])
# int variable for "what is the red-haired person's hairspeed?"
print(s.vars['Red', 'hairspeed'])
# all hairspeed variables associated with different names
print(s.vcats['hairspeed']['name'])

name_color.3.1 in {0,1}
color_hairspeed2 in {1..4}
[name_hairspeed0 in {1..4}, name_hairspeed1 in {1..4}, name_hairspeed2 in {1..4}, name_hairspeed3 in {1..4}, name_hairspeed4 in {1..4}]


Note that these variables are heavily redundant: for example, there is a variable for 'Does Violet have green hair', another for 'Was Violet born under Crabbus', and a third for 'Was the person with green hair born under Crabbus' - when we ask the solver to solve, it will automatically generate a slew of sanity constraints like "If Violet has green hair and Violet was born under Crabbus then the person born under Crabbus must have green hair" and "If Violet has green hair then Violet's hairspeed must match the green-haired person's hairspeed".

We can also provide information via `exclude`, `require`, etc. as above, which will also be translated into solver constraints:

In [8]:
s.exclude("Brita", "Aries") # Brita and Aries cannot be in the same row
s.exclude("Sam", "Green", "Virgo") # No two of these three can be in the same row
s.exclude("name:Violet", "Green") # Violet (the name) cannot be in the same row as Green
s.require("Scorpio", "Blue") # Scorpio and Blue must be in the same row
s.requireOne("Brita", ["Red", "Blue"]) # The row with Brita must contain Red or Blue
s.exclude("Zork", "Crabbus", "Green") # None of these are in the same row

But unlike plain `Grid`, we can now express more complex arithmatic constraints using `s.add` on expressions of vars:

In [9]:
# The red-haired person's hairspeed is twice that of whoever was born under Crabbus
s.add(s.vars['Red','hairspeed'] == 2*s.vars['Crabbus', 'hairspeed'])
# Violet is Taupe-haired or an Aries, but not both
s.add(s.vars['name:Violet','Taupe'] != s.vars['name:Violet','Aries'])
# The sum of all hairspeeds must be between 6 and 10
# Note that we don't have to use 'name' here - we could've used the hairspeed vars
# for any category, as long as we're using the sum of all of them in that category
s.add(sum(s.vcats['hairspeed']['name']) < 10)
s.add(sum(s.vcats['hairspeed']['name']) > 6)


Even more powerfully, we can use `s.add_constraint` to add an arbitrary function that operates on some set of vars. `add_constraint` takes a set of (variable, category) pairs and decorates a function that takes possible values for those pairs; it constrains the solution to only choose values where this function evaluates to `True`.

Thus, to add the constraint that the person with Violet hair must have a longer name than the person associated with Gahoolie, we specify the inputs `('color:Violet', 'name')` (the name of the person with violet hair) and `(Gahoolie', 'name')` (the name of the person born under Gahoolie) and provide a function that takes these two names and returns `True` iff the violet name is longer than the gahoolie name:

In [10]:
@s.add_constraint(('color:Violet', 'name'), ('Gahoolie', 'name'))
def _(vname, gname):
    return len(vname) > len(gname)


Note that under the hood, `Solver` does this by actually checking every single possible combination of values for the specified variables _when you call `add_constraint`_, calling the function on them, and adding constraints to disallow any combination that returned `False`. As a result, calling `add_constraint` with many variables on a grid with a lot of possible values might be quite slow.

Having added this, we can now solve `s` using Numberjack; the returned result contains a completed grid `soln.grid`, and a dataframe of the solution `soln.df`.

In [11]:
soln = s.solve()

In [12]:
soln.grid.html_link()

In [13]:
soln.df

,name,color,sign,hairspeed
0,Brita,Red,Virgo,4
1,Galal,Green,Gahoolie,1
2,Sam,Taupe,Crabbus,2
3,Violet,Violet,Aries,1
4,Zork,Blue,Scorpio,1


## Caveats and future work

### Ambiguity

Right now this just finds the first available solution. If there's more than one (i.e. the problem is underconstrained), it'll find *a* solution but won't tell you that there are others.

### Numberjack

Numberjack, while cool, doesn't seem to be in development any more; sooner or later it'll probably stop working. At some point I'll need to port this to some other combinatorial optimization solver.